In [1]:
import pandas as pd

In [2]:
df_sub = pd.read_csv('data/SampleSubmissionStage1.csv')
df_kp = pd.read_csv('data/Final/final_raw_kp_dat.csv')
df_seed = pd.read_csv('data/Final/final_raw_seeds.csv')

### Prepare Submission file 

To be a test dataset

In [3]:
df_sub[['Season','low_seed','high_seed']] = df_sub['ID'].str.split('_',expand=True).astype(int)

In [4]:
df_sub = pd.merge(df_sub,df_kp,how='inner',
                 left_on=['Season','low_seed'],
                 right_on=['Season','TeamID']).rename(columns={'OrdinalRank':'WKP'})
df_sub = pd.merge(df_sub,df_kp,how='inner',
                 left_on=['Season','high_seed'],
                 right_on=['Season','TeamID']).rename(columns={'OrdinalRank':'LKP'})

In [5]:
df_sub2 = pd.merge(df_sub,df_seed,how='inner',
                  left_on=['Season','low_seed'],
                  right_on=['Season','TeamID']).rename(columns={"Seed": "W_SEED"})
df_sub2 = pd.merge(df_sub2,df_seed,how='inner',
                  left_on=['Season','high_seed'],
                  right_on=['Season','TeamID']).rename(columns={"Seed": "L_SEED"})

In [6]:
df_sub2 = df_sub2.drop(['TeamID_x','TeamID_y'],axis=1)

In [7]:
df_test = df_sub2[['WKP','LKP','W_SEED','L_SEED']]

In [8]:
df_test['Seed_Diff'] = df_test['W_SEED'] - df_test['L_SEED']
df_test['KP_Diff'] = df_test['WKP'] - df_test['LKP']

/Users/meyappansubbaiah/Documents/Github/March_Madness_KG/.env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/meyappansubbaiah/Documents/Github/March_Madness_KG/.env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
df_test_final = df_test[['Seed_Diff','KP_Diff']]

## Build Model

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
import numpy as np

In [11]:
df_train = pd.read_csv('data/Final/data_kp_seed.csv')

In [12]:
y = df_train['result'].values

In [13]:
X = df_train.drop('result',axis=1)

In [14]:
logreg = LogisticRegression(solver='lbfgs')
params = {'C': np.logspace(start=-5, stop=3, num=9)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X, y)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

/Users/meyappansubbaiah/Documents/Github/March_Madness_KG/.env/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Best log_loss: -0.5534, with best C: 0.01


In [15]:
final_reg = LogisticRegression(**clf.best_params_)

In [16]:
final_reg.fit(X,y)

/Users/meyappansubbaiah/Documents/Github/March_Madness_KG/.env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
df_final_sub = pd.read_csv('data/SampleSubmissionStage1.csv')

In [18]:
clipped_preds = np.clip(final_reg.predict_proba(df_test_final)[:,1], 0.05, 0.95)

In [19]:
df_final_sub['Pred'] = clipped_preds

In [20]:
df_final_sub.to_csv('data/Final/Results/logreg_kp_seed.csv',index=False)

In [21]:
df_final_sub

,ID,Pred
0,2014_1107_1110,0.457538
1,2014_1107_1112,0.072169
2,2014_1107_1113,0.084433
3,2014_1107_1124,0.264728
4,2014_1107_1140,0.299164
5,2014_1107_1142,0.822343
6,2014_1107_1153,0.154131
7,2014_1107_1157,0.177657
8,2014_1107_1160,0.700836
9,2014_1107_1163,0.336033
